In [1]:
import cv2
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
dset = tf.keras.utils.image_dataset_from_directory("dset/", batch_size=8000, image_size=(240,240), shuffle=False)

Found 3072 files belonging to 6 classes.


In [3]:
x = np.asarray(list(dset.as_numpy_iterator())[0][0]) / 255
y = np.asarray(tf.one_hot(list(dset.as_numpy_iterator())[0][1], depth=6))

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.9, shuffle=True)

In [5]:
import pickle
import tf2onnx

In [7]:
conv_head = tf.keras.models.load_model("object_recog_head.keras")
spec = (tf.TensorSpec((None, 240, 240, 3), tf.float32, name="input"),)
output_path = "conv_head.onnx"
model_proto = tf2onnx.convert.from_keras(conv_head, input_signature=spec, opset=13, output_path=output_path)

In [5]:
inputs_h = tf.keras.layers.Input(shape=(240, 240, 3))
h = tf.keras.layers.AvgPool2D(pool_size=(3, 3), strides=(3, 3))(inputs_h)
h = tf.keras.layers.Conv2D(16, 4, strides=(2, 2))(h)
h = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(h)
h = tf.keras.layers.Conv2D(32, 5, padding='same')(h)
h = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(h)
h = tf.keras.layers.Conv2D(64, 3, padding='same')(h)
h = tf.keras.layers.Conv2D(64, 3, padding='same')(h)
h = tf.keras.layers.Conv2D(64, 3, padding='same')(h)
h = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(1, 1))(h)
outputs_h = tf.keras.layers.Flatten()(h)
head = tf.keras.models.Model(inputs=inputs_h, outputs=outputs_h)

In [6]:

m = tf.keras.layers.Dense(64, 'relu')(outputs_h)
m = tf.keras.layers.Dense(64, 'relu')(m)
m = tf.keras.layers.Dense(64, 'relu')(m)
m = tf.keras.layers.Dense(64, 'relu')(m)
outputs = tf.keras.layers.Dense(6, 'softmax', name="output_layer")(m)
model = tf.keras.models.Model(inputs=inputs_h, outputs=outputs)


In [ ]:
M1_s = tf.keras.models.Functional([
    tf.keras.layers.AvgPool2D(pool_size=(3, 3), strides=(3, 3), input_shape=(240, 240, 3)),
    tf.keras.layers.Conv2D(16, 4, strides=(2, 2)),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Conv2D(32, 5, padding='same'),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Conv2D(64, 3, padding='same'),
    tf.keras.layers.Conv2D(64, 3, padding='same'),
    tf.keras.layers.Conv2D(64, 3, padding='same'),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(1, 1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, 'relu'),
    tf.keras.layers.Dense(16, 'relu'),
    tf.keras.layers.Dense(16, 'relu'),
    tf.keras.layers.Dense(16, 'relu'),
    tf.keras.layers.Dense(2, 'softmax')
])

In [ ]:
M1 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(96, 12, strides=(4, 4), input_shape=(240, 240, 3)),
    tf.keras.layers.MaxPool2D(pool_size=(4, 4), strides=(2, 2)),
    tf.keras.layers.Conv2D(256, 5, padding='same'),
    tf.keras.layers.MaxPool2D(pool_size=(4, 4), strides=(2, 2)),
    tf.keras.layers.Conv2D(384, 3, padding='same'),
    tf.keras.layers.Conv2D(384, 3, padding='same'),
    tf.keras.layers.Conv2D(256, 3, padding='same'),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, 'relu'),
    tf.keras.layers.Dense(256, 'relu'),
    tf.keras.layers.Dense(2, 'softmax')
])

In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
head.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 240, 240, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 80, 80, 3)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 39, 39, 16)     │           784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 19, 19, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 19, 19, 32)     │        12,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 9, 9, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 9, 9, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 9, 9, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 9, 9, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 105,968 (413.94 KB)

 Trainable params: 105,968 (413.94 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
M1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
print(M1.summary())

In [ ]:
head.trainable = False

In [9]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 240, 240, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 80, 80, 3)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 39, 39, 16)     │           784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 19, 19, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 19, 19, 32)     │        12,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 9, 9, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 9, 9, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 9, 9, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 9, 9, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       200,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 319,606 (1.22 MB)

 Trainable params: 319,606 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 2.9476e-05
Epoch 2/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 2.3000e-05
Epoch 3/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 2.0625e-05
Epoch 4/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 1.8212e-05
Epoch 5/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 1.5149e-05
Epoch 6/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 1.3977e-05
Epoch 7/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 1.0000 - loss: 1.1653e-05
Epoch 8/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 1.0000 - loss: 9.9598e-06
Epoch 9/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 1.0000 - loss: 8.2435e-06
Epoch 10/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 9.6131e-06


In [ ]:
M1.fit(x_train, y_train, epochs=50)

In [ ]:
M1_s.evaluate(x_test, y_test)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
M1.evaluate(x_test, y_test)

In [ ]:
import time

In [ ]:
import socket

In [ ]:
cam = cv2.VideoCapture(0)
key = ' '
while key != 113:
    ret, frame = cam.read()
    frame = cv2.resize(frame[:,80:560], (240, 240))
    infer = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    start = time.time()
    res = model.predict(np.asarray(infer).reshape(1, 240, 240, 3))
    infer_time = time.time() - start
    print(res)
    color = (0, 255, 0)
    if res[0][1] < res[0][0]:
        color = (0, 0, 255)
    cv2.imshow("Hello", cv2.rectangle(frame, (0, 0), (239, 239), color, 5))

    key = cv2.waitKey(10)

cam.release()
cv2.destroyAllWindows()

In [13]:
head.save("object_recog_head_v2.keras")

In [ ]:

host = '192.168.1.94'  # Server IP address
port = 3333
data = b""
# Create socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect((host, port))
key = ' '
while key != 113:

    msg_size = int.from_bytes(client_socket.recv(4), 'little')
    to_receive = msg_size
    #print(to_receive)
    while len(data) < msg_size:
        data += client_socket.recv(min(to_receive, 512))
        to_receive = msg_size - len(data)


    frame_data = data[:msg_size]
    data = data[msg_size:]

    # Decode JPEG format
    frame = cv2.imdecode(np.frombuffer(frame_data, np.uint8), cv2.IMREAD_COLOR)
    infer = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    start = time.time()
    res = model.predict(np.asarray(infer).reshape(1, 240, 240, 3))
    infer_time = time.time() - start
    print(infer_time)
    color = (0, 255, 0)
    if res[0][1] < res[0][0]:
        color = (0, 0, 255)
    cv2.imshow("Hello", cv2.rectangle(frame, (0, 0), (239, 239), color, 5))
    key = cv2.waitKey(10)

# Release resources
client_socket.close()
cv2.destroyAllWindows()

In [ ]:
import tf2onnx
import pickle

In [ ]:
spec = (tf.TensorSpec((None, 240, 240, 3), tf.float32, name="input"),)
output_path = "person.onnx"
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)

In [ ]:
model

In [ ]:
import math

In [ ]:
with open("person.pickle", "wb") as f:
    pickle.dump((x_test, y_test), f)

In [ ]:
with open("script.txt", "wt") as f_2:
    f_2.write("__attribute__((aligned(8))) int8_t example_element[] = {")
    for row in x_test[1]:
        for col in row:
            for val in col:
                f_2.write(str(max(min(int(val * int(1 << 7)), 127), -128)) + ", ")
        

In [ ]:
print(y_test[1])